In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

sns.set(style="whitegrid")
%matplotlib inline

In [2]:
train = pd.read_csv("data/train.csv", index_col="PassengerId")
test = pd.read_csv("data/test.csv", index_col="PassengerId")
train.shape, test.shape

((891, 11), (418, 10))

In [3]:
train.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
label_name = "Survived"

In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X_train = ohe.fit_transform(train.drop(label_name, axis=1)).toarray()
X_test = ohe.transform(test).toarray()
# 데이터 로딩
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

X_train.shape

torch.Size([891, 2080])

In [6]:
# Label Encoding
y_train = train[[label_name]]
y_train = torch.Tensor(y_train.values)
y_train[:5]

tensor([[0.],
        [1.],
        [1.],
        [1.],
        [0.]])

In [7]:
y_train.shape

torch.Size([891, 1])

In [8]:
# 모델 정의
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [9]:
# 모델 초기화
input_size = X_train.shape[1]
model = LogisticRegression(input_size)

# 손실 함수 및 optimizer 설정
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습
num_epochs = 1000
for epoch in range(num_epochs):
    # forward + backward + optimize
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # 중간 결과 출력
    if (epoch+1) % 50 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [50/1000], Loss: 0.6446
Epoch [100/1000], Loss: 0.6152
Epoch [150/1000], Loss: 0.5956
Epoch [200/1000], Loss: 0.5803
Epoch [250/1000], Loss: 0.5673
Epoch [300/1000], Loss: 0.5560
Epoch [350/1000], Loss: 0.5460
Epoch [400/1000], Loss: 0.5371
Epoch [450/1000], Loss: 0.5290
Epoch [500/1000], Loss: 0.5218
Epoch [550/1000], Loss: 0.5152
Epoch [600/1000], Loss: 0.5093
Epoch [650/1000], Loss: 0.5039
Epoch [700/1000], Loss: 0.4990
Epoch [750/1000], Loss: 0.4946
Epoch [800/1000], Loss: 0.4905
Epoch [850/1000], Loss: 0.4867
Epoch [900/1000], Loss: 0.4833
Epoch [950/1000], Loss: 0.4801
Epoch [1000/1000], Loss: 0.4771


* torch.no_grad()는 PyTorch에서 gradient 계산을 수행하지 않도록 하는 context manager입니다. 이를 사용하면 모델의 inference 과정에서 gradient 계산을 하지 않아 메모리 사용량을 줄일 수 있습니다. 또한 gradient 계산이 필요하지 않은 validation, test 데이터셋 등에서 사용하여 불필요한 계산을 방지할 수 있습니다.


In [10]:
# 테스트
with torch.no_grad():
    outputs = model(X_test)
    predictions = (outputs >= 0.5).float()

predictions[:5]

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [1.]])

In [11]:
pd.Series(predictions.squeeze()).value_counts()

0.0    297
1.0    121
dtype: int64